<h1>Assignment-4</h1>
<h2>Conditional Random Fields</h2>

In [4]:
#installing sklearn-crfsuite
!pip install sklearn-crfsuite

In [28]:
#importing necessary libraries
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import tensorflow as tf
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics



<b>Preprocessing<b>

In [29]:
train_dataset = list(open("hi-ud-train.conllu", "r", encoding="utf-8"))
test_dataset = list(open("hi-ud-test .conllu", "r", encoding="utf-8"))
#print(train_dataset)
#print(test_dataset)
train_data = []
test_data= []
sentence=[]
for val in train_dataset[1:]: 
    val_list=val[:-1].split(',')  
    #print(val_list)
    if val_list[0]!='':       
        sentence.append(val_list)        
    else:
        train_data.append(sentence)        
        sentence=[]
        
sentence=[]
for val in test_dataset[1:]: 
    val_list=val[:-1].split('\t')  
    #print(val_list)
    if val_list[0]!='':       
        sentence.append(val_list)        
    else:
        test_data.append(sentence)        
        sentence=[]

        
        
#print(train_data[0])
#print(test_data[0])

<b>Feature Extraction</b>

In [46]:
def word2features(sent, i):
    word = sent[i][1]
    postag = sent[i][2]

    features = {        
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [postag for iD, word, postag in sent]

def sent2tokens(sent):
    return [word for iD, word, postag in sent]



In [47]:
X_train = [sent2features(s) for s in train_data]
y_train = [sent2labels(s) for s in train_data]

X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]

In [48]:
from sklearn_crfsuite import CRF
 
model = CRF()
model.fit(X_train, y_train)
labels = list(model.classes_)

In [49]:

labels

['DET',
 'PROPN',
 'ADP',
 'ADV',
 'ADJ',
 'NOUN',
 'NUM',
 'AUX',
 'PUNCT',
 'PRON',
 'VERB',
 'CCONJ',
 'PART',
 'COMMA',
 'SCONJ',
 'X']

<b>Most likely Transitions</b>

In [50]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

        
modeltest= CRF()
modeltest.fit(X_test,y_test)
print("------------------------Train set---------------------")
print("--------------Top 10 likely transitions:--------------")
print_transitions(Counter(model.transition_features_).most_common(10))
print("---------------------------------------------------")
print("\n------------Top 10 unlikely transitions:--------------")
print_transitions(Counter(model.transition_features_).most_common()[-10:])
print("-----------------------------------------------------")
print('\n')
print("------------------------Test set----------------------")
print("--------------Top 10 likely transitions:--------------")
print_transitions(Counter(modeltest.transition_features_).most_common(10))
print("------------------------------------------------------")
print("\n------------Top 10 unlikely transitions:--------------")
print_transitions(Counter(modeltest.transition_features_).most_common()[-10:])
print("-------------------------------------------------------")

------------------------Train set---------------------
--------------Top 10 likely transitions:--------------
VERB   -> AUX     3.087001
ADJ    -> NOUN    2.753777
PROPN  -> PROPN   2.391956
NOUN   -> ADP     2.237005
PROPN  -> ADP     2.133795
DET    -> NOUN    1.809328
NUM    -> NOUN    1.714033
PRON   -> ADP     1.640931
ADJ    -> VERB    1.618387
NOUN   -> VERB    1.582617
---------------------------------------------------

------------Top 10 unlikely transitions:--------------
ADP    -> X       -0.558948
CCONJ  -> VERB    -0.559135
DET    -> PROPN   -0.660447
AUX    -> VERB    -0.692009
PROPN  -> DET     -0.774447
ADP    -> SCONJ   -0.795802
ADP    -> AUX     -0.813619
AUX    -> ADJ     -0.851825
ADP    -> COMMA   -0.853062
ADP    -> CCONJ   -1.038824
-----------------------------------------------------


------------------------Test set----------------------
--------------Top 10 likely transitions:--------------
VERB   -> AUX     2.300905
ADJ    -> NOUN    1.916280
PROPN  -> PR

<b>Predictions</b>

In [51]:
from sklearn_crfsuite import metrics
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

print("-----------------------Accuracy----------------------")
print("                 ",metrics.flat_accuracy_score(y_test, y_pred))
print("-----------------------------------------------------")


sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:2])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))
print("-----------------------------------------------------")


-----------------------Accuracy----------------------
                  0.8005540166204986
-----------------------------------------------------
              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      1.000     0.848     0.918        33
       CCONJ      1.000     0.960     0.980        25
       SCONJ      0.500     1.000     0.667         3
         ADP      0.940     0.937     0.939       303
         ADV      0.600     0.286     0.387        21
         ADJ      0.558     0.713     0.626        94
         DET      0.897     0.722     0.800        36
        VERB      0.713     0.727     0.720        99
        NOUN      0.732     0.809     0.768       324
       COMMA      0.000     0.000     0.000         0
       PROPN      0.603     0.528     0.563       144
        PRON      0.817     0.754     0.784        65
         NUM      0.950     0.760     0.844        25
         AUX      0.866     0.935     0.899 